In [1]:
from pymongo import MongoClient
from pymongo.encryption import AutoEncryptionOpts
from dotenv import load_dotenv
import os
import pprint
import datetime

In [ ]:
# setup some variables
load_dotenv()
uri = os.environ['MONGODB_URI']
key_vault_database_name = "encryption"
key_vault_collection_name = "__keyVault_customers"
key_vault_namespace = f"{key_vault_database_name}.{key_vault_collection_name}"
encrypted_database_name = "sample_analytics"
encrypted_collection_name = "customers_enc"

In [ ]:
# build local kms provider credentials with the local CMK
try:
    path = "./customer-master-key.txt"
    with open(path, "rb") as f:
        local_master_key = f.read()
        kms_provider_credentials = {
            "local": {
                "key": local_master_key
            },
        }
except Exception as e:
    raise Exception("Unable to read Customer Master Key from file due to the following error: ", e)

# build MongoDB clients (standard one and encrypted)
encrypted_client = MongoClient(
    uri,
    auto_encryption_opts= AutoEncryptionOpts(
        kms_provider_credentials,
        key_vault_namespace,
        crypt_shared_lib_required=True,
        crypt_shared_lib_path=os.environ['SHARED_LIB_PATH']
    )
)

standard_client = MongoClient(
    uri)

# get the encrypted collection with the two clients
encrypted_collection = encrypted_client[encrypted_database_name][encrypted_collection_name]
collection = standard_client[encrypted_database_name][encrypted_collection_name]

### Insert document with unconfigured client

In [ ]:
try:
    collection.insert_one({
        'accounts': [550665, 321695],
        'address': '20 rue Quentin Bauchart\n 75008 PARIS',
        'birthdate': datetime.datetime(1978, 5, 11, 0, 0),
        'email': 'john.doe@mongodb.com',
        'name': 'John Doe',
        'tier_and_details': {},
        'username': 'whoami'
    })
except Exception as e:
    print("Unable to insert document (as expected) with error: ", e)

### Insert document

In [ ]:
encrypted_collection.insert_one({
    'accounts': [550665, 321695],
    'address': '20 rue Quentin Bauchart\n 75008 PARIS',
    'birthdate': datetime.datetime(1978, 5, 11, 0, 0),
    'email': 'john.doe@mongodb.com',
    'name': 'John Doe',
    'tier_and_details': {},
    'username': 'whoami'
})

### Query on unencrypted String field (with unconfigured client)

In [ ]:
find_result = collection.find_one({
    "username": "whoami"
},
{"__safeContent__": 0})
pprint.pprint(find_result)

### Query on encrypted String field (with unconfigured client)

In [ ]:
find_result = collection.find_one({
    "name": "John Doe"
},
{"__safeContent__": 0})
pprint.pprint(find_result)

### Query on encrypted String field

In [ ]:
find_result = encrypted_collection.find_one({
    "name": "John Doe"
},
{"__safeContent__": 0})
pprint.pprint(find_result)

### Query on Date field

In [ ]:
date_param = datetime.datetime(1988, 6, 20, 0, 0)
find_result = encrypted_collection.find_one({
   "birthdate": date_param 
},
{"__safeContent__": 0})
pprint.pprint(find_result)

###  Query on Boolean field with $in operator

In [ ]:
find_result = encrypted_collection.find_one({
  "active": { "$in": [ True ] } 
},
{"__safeContent__": 0})
pprint.pprint(find_result)


### Query on unencrypted and encrypted String fields ($and)

In [ ]:
find_result = encrypted_collection.find_one({
    "$and" : [ { "name": "John Doe" }, { "username": "whoami" } ]
},
{"__safeContent__": 0})
pprint.pprint(find_result)

### Query on two encrypted String fields ($and)

In [ ]:
find_result = encrypted_collection.find_one({
    "$and" : [ { "name": "John Doe" }, { "email": "john.doe@mongodb.com" } ]
},
{"__safeContent__": 0})
pprint.pprint(find_result)